In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn
from tqdm import tqdm 

In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device('cpu')
    print("GPU not available, using CPU.")

Using GPU: NVIDIA RTX A5000


In [5]:
transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225])])
train_data = datasets.ImageFolder(root='archive//ASL_Dataset//Train',transform=transform)

In [6]:
train_data = DataLoader(train_data, batch_size=16,shuffle=True)

In [7]:
train_data.dataset.classes

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'Nothing',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'Space',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']

In [8]:
import numpy as np
import plotly.express as px

# Get a batch iterator
data_iter = iter(train_data)

# Display first 10 images
for i in range(10):
    images, labels = next(data_iter)
    
  
    # Get the first image in the batch and denormalize
    image = images[0].numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean  # Denormalize
    image = np.clip(image, 0, 1)

    # Plot using Plotly
    fig = px.imshow(image, title=f"Label: {labels[0].item()}")
    fig.update_layout(coloraxis_showscale=False)
    fig.show()



In [ ]:
from collections import Counter

label_list = [labels for _, labels in train_data.dataset]

class_counts = Counter(label_list)

# Create a bar plot
px.bar(x=list(class_counts.keys()),y=list(class_counts.values()),title="Class Distribution").show()

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3,16,kernel_size=3, stride=1,padding=1),
            nn.ReLU(),
            nn.Conv2d(16,32,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2,stride=2),
            nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64 * 112 * 112, 512),
            nn.Linear(512,28),
            #nn.Softmax(dim=1)

        )

    def forward(self,x):
        return self.model(x)

In [ ]:
model = CNN()

In [ ]:

print(f"Using device: {device}")
model = model.to(device)


Using device: cpu


In [ ]:
cross = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 5

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    running_correct = 0
    total = 0
    for images, labels in tqdm(train_data):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(images)
        loss = cross(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predict = torch.max(output.data, 1)
        running_correct += (predict == labels).sum().item()
        total += labels.size(0)
    epoch_loss = running_loss / len(train_data)
    accu = running_correct / total
    print(f"{epoch+1}/{epoch} loss: {epoch_loss:.4f} acc: {accu:.4f}")


torch.save(model.state_dict(), 'model.pth')

  0%|          | 0/10355 [00:02<?, ?it/s]


RuntimeError: MPS backend out of memory (MPS allocated: 8.67 GB, other allocations: 34.83 MB, max allowed: 9.07 GB). Tried to allocate 1.53 GB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:

test_data = datasets.ImageFolder(root='archive//ASL_Dataset//Test', transform=transform)
test_data = DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
model.eval()

correct = 0
total_test = 0
test_loss = 0.0


with torch.no_grad():
    for images, labels in tqdm(test_data):
        images, labels = images.to(device), labels.to(device)
        out = model(images)
        loss = cross(out, labels)
        test_loss += loss.item()
        _, predict = torch.max(out.data, 1)
        correct += (predict == labels).sum().item()
        total_test += labels.size(0)

    accu = correct / total_test
    avg_loss = test_loss / len(test_data)
    print(f"Accuracy: {accu:.4f}, Loss: {avg_loss:.4f}")
